Imports

In [4]:
import pandas as pd
import numpy as np

Load csv files into dataframes

In [5]:
df_clicks = pd.read_csv('clicks.csv', ',')
df_impressions = pd.read_csv('impressions.csv')

c:\Users\lacal\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3397: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_clicks

,clickTimestamp,impressionId
0,2021-04-20 10:50:07,1
1,2021-04-20 10:59:48,3
2,2021-04-20 11:23:21,4
3,2021-04-20 11:18:45,5
4,2021-04-21 00:04:58,11
5,2021-04-21 12:11:07,14
6,2021-04-21 12:20:26,14
7,2021-04-21 12:23:34,15
8,2021-04-21 17:12:48,22
9,2021-04-21 17:26:53,22


Merge 'clicks' and 'impressions' into new dataframe

In [7]:
df = pd.merge(df_impressions, df_clicks, on="impressionId", how='right')

Sort newly created dataframe by 'adId', 'visitorHash' and 'clickTimestamp'

In [8]:
df = df.sort_values(by = ['adId', 'visitorHash', 'clickTimestamp'], na_position = 'first')
df['clickTimestamp'] = pd.to_datetime(df['clickTimestamp'], format="%Y-%m-%d %H:%M:%S")
df = df.reset_index()

When its sorted, we can see individual clicks of users on advertisements

In [9]:
df.head(12)

,index,impressionTime,impressionId,adId,visitorHash,clickTimestamp
0,8,2021-04-21 17:07:08,22,1,qwFm9BDFVR9UigCwRiwmNSwlJZkP3PyU,2021-04-21 17:12:48
1,9,2021-04-21 17:07:08,22,1,qwFm9BDFVR9UigCwRiwmNSwlJZkP3PyU,2021-04-21 17:26:53
2,10,2021-04-21 23:58:07,26,2,8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7,2021-04-21 23:59:35
3,11,2021-04-22 00:04:56,27,2,8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7,2021-04-22 00:10:23
4,4,2021-04-21 00:02:43,11,2,ywXGzqthzpkiwsrTckExu9nVVoDjNjK4,2021-04-21 00:04:58
5,3,2021-04-20 11:09:45,5,3,8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7,2021-04-20 11:18:45
6,2,2021-04-20 11:03:12,4,3,PFm4s8dmTBsIFEMqHApcGgutH2goD0v2,2021-04-20 11:23:21
7,0,2021-04-20 10:48:07,1,4,8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7,2021-04-20 10:50:07
8,1,2021-04-20 10:52:57,3,4,8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7,2021-04-20 10:59:48
9,5,2021-04-21 12:07:51,14,4,PFm4s8dmTBsIFEMqHApcGgutH2goD0v2,2021-04-21 12:11:07


Remove duplicate clicks

In [10]:
maxSeconds = 600 # 10 minutes in seconds
previousIndex = 0
for i, row in df.iterrows():

  # we don't look at the first line because we have nothing to compare it with
  if (i != 0):

    # boolean variable, which is used to identify the deleted click,
    # after deleting we do not want to compare the next line with the deleted
    deleted = False

    # we compare the click only if the hash of the user and the advertisement are equal
    if ((df.at[i, 'visitorHash'] == df.at[previousIndex, 'visitorHash']) and (df.at[i, 'adId'] == df.at[previousIndex, 'adId'])):

        # subtract click times and get the difference in seconds
        subtract = (df.at[i, 'clickTimestamp'] - df.at[previousIndex, 'clickTimestamp']).total_seconds()

        # if the difference is less than 10 minutes (600 seconds), then we will delete the row from the dataframe
        if (subtract < maxSeconds):
          df_clicks.drop(df.at[i, 'index'], inplace=True)
          deleted = True
    
    # if the row has not been deleted, set the previous row index to current
    if(not deleted):
      previousIndex = i
      deleted = False

We removed 2 rows from the table, where the same user clicked twice on the same advertisement in less than 10 minutes

In [11]:
df_clicks.head(100)

,clickTimestamp,impressionId
0,2021-04-20 10:50:07,1
2,2021-04-20 11:23:21,4
3,2021-04-20 11:18:45,5
4,2021-04-21 00:04:58,11
5,2021-04-21 12:11:07,14
7,2021-04-21 12:23:34,15
8,2021-04-21 17:12:48,22
9,2021-04-21 17:26:53,22
10,2021-04-21 23:59:35,26
11,2021-04-22 00:10:23,27


MySQL - docker

![title](docker_container.PNG)

Create a new database if it doesn't exist

In [12]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")
exists = False

for x in mycursor:
  if (str(x) == "('seznam_advertising',)"): exists = True

if (not exists):
  mycursor.execute("CREATE DATABASE seznam_advertising")

Create tables if they do not exist in the database

In the assignment is written to create a table, but it doesn't seem appropriate to me to combine impressions and clicks, as we will have empty values when the impression had no clicks

In [14]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root",
  database="seznam_advertising"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
exists = False

for x in mycursor:
  if (str(x) == "('clicks',)" or str(x) == "('impressions',)" ): exists = True

if (not exists):
    mycursor.execute("CREATE TABLE impressions (impressionId INT PRIMARY KEY, impressionTime DATETIME, adId INT, visitorHash VARCHAR(255))")
    mycursor.execute("CREATE TABLE clicks (id INT AUTO_INCREMENT PRIMARY KEY, clickTimestamp DATETIME, impressionId INT, FOREIGN KEY (impressionId) REFERENCES impressions(impressionId))")

We will use create_engine from sqlalchemy to insert dataframes into database

In [15]:
from sqlalchemy import create_engine

DATABSE_URI='mysql+mysqlconnector://{user}:{password}@{server}/{database}'.format(user='root', password='root', server='127.0.0.1', database='seznam_advertising')

engine = create_engine(DATABSE_URI)

Insert dataframe 'impressions' into database

In [16]:
df_impressions.to_sql('impressions', engine, if_exists='append', index=False)  

30

Insert dataframe 'clicks' into databázy

In [17]:
df_clicks.to_sql('clicks', engine, if_exists='append', index=False)  

10

Load inserted rows from table 'impressions'

In [18]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root",
  database="seznam_advertising"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM impressions")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(1, datetime.datetime(2021, 4, 20, 10, 48, 7), 4, '8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7')
(2, datetime.datetime(2021, 4, 20, 10, 54, 45), 1, 'qwFm9BDFVR9UigCwRiwmNSwlJZkP3PyU')
(3, datetime.datetime(2021, 4, 20, 10, 52, 57), 4, '8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7')
(4, datetime.datetime(2021, 4, 20, 11, 3, 12), 3, 'PFm4s8dmTBsIFEMqHApcGgutH2goD0v2')
(5, datetime.datetime(2021, 4, 20, 11, 9, 45), 3, '8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7')
(6, datetime.datetime(2021, 4, 20, 11, 45, 6), 1, 'ywXGzqthzpkiwsrTckExu9nVVoDjNjK4')
(7, datetime.datetime(2021, 4, 20, 11, 56, 12), 2, 'qwFm9BDFVR9UigCwRiwmNSwlJZkP3PyU')
(8, datetime.datetime(2021, 4, 20, 13, 6, 50), 1, '8VTYLnaJZW7dfQH72tPXAbvyNIWeWLU7')
(9, datetime.datetime(2021, 4, 20, 14, 2, 12), 4, 'qwFm9BDFVR9UigCwRiwmNSwlJZkP3PyU')
(10, datetime.datetime(2021, 4, 20, 23, 52, 59), 2, 'ywXGzqthzpkiwsrTckExu9nVVoDjNjK4')
(11, datetime.datetime(2021, 4, 21, 0, 2, 43), 2, 'ywXGzqthzpkiwsrTckExu9nVVoDjNjK4')
(12, datetime.datetime(2021, 4, 21, 6, 45, 51), 1

Load inserted rows from table 'clicks'

In [19]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root",
  database="seznam_advertising"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM clicks")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('2021-04-20 10:50:07', 1)
('2021-04-20 11:23:21', 4)
('2021-04-20 11:18:45', 5)
('2021-04-21 00:04:58', 11)
('2021-04-21 12:11:07', 14)
('2021-04-21 12:23:34', 15)
('2021-04-21 17:12:48', 22)
('2021-04-21 17:26:53', 22)
('2021-04-21 23:59:35', 26)
('2021-04-22 00:10:23', 27)


Queries

How many impressions did the ad with ID X had on day Y?

In [20]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root",
  database="seznam_advertising"
)

mycursor = mydb.cursor()

startDate = '2021-04-20 00:00:00'
endDate = '2021-04-21 00:00:00'
adId = "4"
mycursor.execute("SELECT COUNT(*) FROM impressions WHERE adId = " + adId + " AND impressionTime >='" + startDate + "' AND impressionTime <'" + endDate + "'")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(3,)


How many clicks were there on day Y?


In [28]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="root",
  database="seznam_advertising"
)

mycursor = mydb.cursor()

startDate = '2021-04-20 00:00:00'
endDate = '2021-04-21 00:00:00'
mycursor.execute("SELECT COUNT(*) FROM `clicks` INNER JOIN `impressions` ON impressions.impressionId = clicks.impressionId WHERE impressionTime >='" + startDate + "' AND impressionTime <'" + endDate + "'")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)

(6,)
